In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/

/content/drive


In [ ]:
cd MyDrive/

/content/drive/MyDrive


In [ ]:
cd cgcnn_hoip/

/content/drive/MyDrive/cgcnn_hoip


In [ ]:
!pip3 install pymatgen

In [ ]:
from random import sample
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
from torch.autograd import Variable
from torch.utils.data import DataLoader
from cgcnn.data import CIFData
from cgcnn.data import collate_pool, get_train_val_test_loader
from cgcnn.model import CrystalGraphConvNet
gpu_boole = torch.cuda.is_available()

In [ ]:
# The code for the Normalizer and AverageMeter class was taken from the
# original CGCNN repoistory
# The dataset and model was also taken from the original model and slightly modified

In [ ]:
class Normalizer(object):
    """Normalize a Tensor and restore it later. """
    def __init__(self, tensor):
        """tensor is taken as a sample to calculate the mean and std"""
        self.mean = torch.mean(tensor)
        self.std = torch.std(tensor)

    def norm(self, tensor):
        return (tensor - self.mean) / self.std

    def denorm(self, normed_tensor):
        return normed_tensor * self.std + self.mean

    def state_dict(self):
        return {'mean': self.mean,
                'std': self.std}

    def load_state_dict(self, state_dict):
        self.mean = state_dict['mean']
        self.std = state_dict['std']

In [ ]:
def mae(prediction, target):
    """
    Computes the mean absolute error between prediction and target
    Parameters
    ----------
    prediction: torch.Tensor (N, 1)
    target: torch.Tensor (N, 1)
    """
    return torch.mean(torch.abs(target - prediction))

def rmse(prediction, target):
    """
    Computes the root mean square error between prediction and target
    Parameters
    ----------
    prediction: torch.Tensor(N, 1)
    target: torch.Tensor(N, 1)
    """

    return torch.sqrt(torch.mean(torch.square(target - prediction)))

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
dataset = CIFData('cif_merge')
collate_fn = collate_pool
train_loader, val_loader, test_loader = get_train_val_test_loader(dataset=dataset, collate_fn=collate_fn, batch_size=5, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, pin_memory=gpu_boole, return_test=True, train_size=None, test_size=None, val_size=None)
structures, _, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]
print(orig_atom_fea_len, nbr_fea_len)

92 41


/usr/local/lib/python3.6/dist-packages/pymatgen/io/cif.py:745: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.6/dist-packages/pymatgen/io/cif.py:1194: UserWarning:

Issues encountered while parsing CIF: No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.



In [ ]:
model_checkpoint = torch.load('band-gap.pth.tar')

In [ ]:
print(model_checkpoint['args']['atom_fea_len'])
# print(model_checkpoint['state_dict'].keys())
model_args = model_checkpoint['args']
print(model_args)

64
{'dataset': 'matproj', 'data_options': ['/oasis/scratch/comet/txie/temp_project/data/matproj/20170419_semi.json', 'regression', 'band_gap'], 'atom_encoder': 'custom', 'atom_encoder_param': ['/home/txie/works/cgnn/Local/embeddings/elem_prop_embedding.json'], 'task': 'regression', 'n_out': 1, 'disable_cuda': False, 'workers': 0, 'epochs': 1000, 'start_epoch': 0, 'batch_size': 256, 'lr': 0.05, 'lr_milestones': [800], 'momentum': 0.9, 'weight_decay': 0.0, 'print_freq': 10, 'resume': '', 'train_size': 16458, 'val_size': 5485, 'test_size': 5486, 'optim': 'SGD', 'atom_fea_len': 64, 'h_fea_len': 32, 'n_conv': 4, 'n_h': 1, 'cuda': True}


In [ ]:
model = CrystalGraphConvNet(orig_atom_fea_len, nbr_fea_len,
                            atom_fea_len=model_args['atom_fea_len'],
                            n_conv=model_args['n_conv'], h_fea_len=model_args['h_fea_len'], n_h=model_args['n_h'],
                            classification=False)
# aaron: forgot h_fea_len when initialize the model
# print(model)
# print(model_checkpoint['state_dict'])

if gpu_boole:
    model.cuda()
criterion=nn.MSELoss()
normalizer = Normalizer(torch.zeros(3))
# normalizer.load_state_dict = model_checkpoint['normalizer']
# normalizer.load_state_dict = model_checkpoint['normalizer']
# aaron: load_state_dict(state) not equal sign (perhaps)
model.load_state_dict(model_checkpoint['state_dict'])
normalizer.load_state_dict(model_checkpoint['normalizer'])


In [ ]:
model.eval()
mae_errors = AverageMeter()
losses = []
for i, (input, target, batch_cif_ids) in enumerate(test_loader):
    with torch.no_grad():
        if gpu_boole:
            input_var = (input[0].cuda(),
                         input[1].cuda(),
                         input[2].cuda(),
                         [crys_idx.cuda() for crys_idx in input[3]])
        else:
            input_var = input
        target_normed = normalizer.norm(target)
        if gpu_boole:
            target_normed = target_normed.cuda()
        output = model(*input_var)
        loss = criterion(output, target_normed)
        losses.append(loss)
        error = mae(normalizer.denorm(output.data.cpu()), target)
        mae_errors.update(error, target.size(0))


/usr/local/lib/python3.6/dist-packages/pymatgen/io/cif.py:745: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.6/dist-packages/pymatgen/io/cif.py:1194: UserWarning:

Issues encountered while parsing CIF: No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.6/dist-packages/pymatgen/io/cif.py:1194: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.



In [ ]:
print(loss)

tensor(1.1104, device='cuda:0')


In [ ]:
print(mae_errors.avg)
# 0.8259 (eV? I suppose) for the mae_errors, probably too high

tensor(0.7716)


In [ ]:
print(losses)

[tensor(0.3083, device='cuda:0'), tensor(0.4376, device='cuda:0'), tensor(0.6844, device='cuda:0'), tensor(0.3267, device='cuda:0'), tensor(0.4226, device='cuda:0'), tensor(0.4608, device='cuda:0'), tensor(0.1470, device='cuda:0'), tensor(0.2617, device='cuda:0'), tensor(0.3242, device='cuda:0'), tensor(0.3099, device='cuda:0'), tensor(0.3504, device='cuda:0'), tensor(0.1298, device='cuda:0'), tensor(0.2839, device='cuda:0'), tensor(0.5406, device='cuda:0'), tensor(0.3156, device='cuda:0'), tensor(0.7301, device='cuda:0'), tensor(0.0212, device='cuda:0'), tensor(0.4108, device='cuda:0'), tensor(0.2868, device='cuda:0'), tensor(0.0725, device='cuda:0'), tensor(0.6338, device='cuda:0'), tensor(0.3677, device='cuda:0'), tensor(0.3140, device='cuda:0'), tensor(0.2582, device='cuda:0'), tensor(0.5439, device='cuda:0'), tensor(0.4665, device='cuda:0'), tensor(0.1823, device='cuda:0'), tensor(0.2920, device='cuda:0'), tensor(0.4995, device='cuda:0'), tensor(0.3368, device='cuda:0'), tensor(0.